In [1]:
import numpy as np
import pandas as pd
from textblob import TextBlob
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import warnings

warnings.filterwarnings("ignore")

In [62]:
df = pd.read_csv('Movie-2023-06-15.csv')[['description','name']]
df.columns = ['Synopsis','Movie_Name']
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def cleandata(text):
    tag_mapping = {
    'NN': 'n',
    'NNS': 'n',
    'NNP': 'n',
    'NNPS': 'n',
    'VB': 'v',
    'VBD': 'v',
    'VBG': 'v',
    'VBN': 'v',
    'VBP': 'v',
    'VBZ': 'v',
    'JJ': 'a',
    'JJR': 'a',
    'JJS': 'a',
    'RB': 'r',
    'RBR': 'r',
    'RBS': 'r',
    'WRB':'r'
}
    text = "".join([word.lower() for word in re.sub(r"[^a-zA-Z]", " ", text)])
    tokens = [str(TextBlob(word).correct()) for word in word_tokenize(text)]
    text = [word for word in tokens if len(word)>2 or word not in stop_words]
    text = " ".join([lemmatizer.lemmatize(word, pos='n' if tag_mapping.get(TextBlob(word).tags[0][1]) == None else tag_mapping.get(TextBlob(word).tags[0][1])) for word in text])
    return text

In [ ]:
df.head(2)

,Synopsis,Movie_Name
0,An orphaned boy enrolls in a school of wizardr...,Harry Potter and the Sorcerer's Stone
1,"An American agent, under false suspicion of di...",Mission: Impossible


In [19]:
df1 = df.copy()
df1['Synopsis'] = df1['Synopsis'].apply(lambda x: cleandata(x))
df1

,Synopsis,Movie_Name
0,orphan boy enrol school wizard where learns th...,Harry Potter and the Sorcerer's Stone
1,american agent under false suspicion disloyalt...,Mission: Impossible
2,don veto corleone head mania family decides ha...,The Godfather
3,harry and heroine search for voldemort remain ...,Harry Potter and the Deathly Hallows: Part 2
4,decade after survive the kostroma incident ell...,Aliens
...,...,...
84,the story e lawrence the english officer who s...,Lawrence of Arabia
85,after refuse attack enemy position general acc...,Paths of Glory
86,after be kidnap and imprison for fifteen year ...,Oldboy
87,the rise arthur fleck from aspirin stand comed...,Joker


In [80]:
from fuzzywuzzy import fuzz

def find_movie_by_description(description, dataset):
    best_match = []

    for index, row in dataset.iterrows():
        ratio = fuzz.token_set_ratio(description, row['Synopsis'])
        if ratio > 90:
            best_match.append(row['Movie_Name'])

    return best_match

# Example usage
description = input("Enter the Movie Discription: ")
result = find_movie_by_description(description, df1)
if len(result) >0:
    print(result)
else:
    print('No records found')


["Harry Potter and the Sorcerer's Stone", 'Harry Potter and the Deathly Hallows: Part 2']
